In [28]:
import json
from langdetect import detect
from tqdm.notebook import tqdm

In [29]:
with open('./data/evjvqa_train_lang.json', 'r', encoding='utf-8') as f:
    data = json.load(f)
    
annotations = data['annotations']

In [30]:
len(annotations)

23785

In [31]:
lang_count = {
    "vi": 0,
    "en": 0,
    "ja": 0
}

for anno in tqdm(annotations):
    question = anno['question']
#     lang = detect(question)
#     anno['language'] = lang
    lang_count[anno['language']] += 1
        
lang_count

  0%|          | 0/23785 [00:00<?, ?it/s]

{'vi': 8332, 'en': 7194, 'ja': 8259}